# 1: Import essential libraries

In [1]:
import sys
sys.path.append('/home/quangminh/Documents/code/Python/kfot/notebooks')

In [2]:
%cd ..

/home/quangminh/Documents/code/Python/kfot


In [4]:
# from transformers import CLIPModel,CLIPProcessor
# import torch
# from PIL import Image
# import ot
# from random import randint

# Disable warnings
import warnings
warnings.filterwarnings('ignore')

from optimal_transport.experiments.vision_language.datasets import *

from tqdm import tqdm

# 2: Embedding images and texts

In [6]:
# Model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [5]:
test_data = cifar10()

Files already downloaded and verified


In [8]:
# Precompute one text embedding per class
class_text_embeds = {}
for class_name in tqdm(test_data.classes):
    text = "A photo of " + class_name
    text_input = processor(text=text, return_tensors='pt', padding=True)
    with torch.no_grad():
        text_embeds = model.get_text_features(**text_input)  # Get the text embedding

        text_embeds = text_embeds / text_embeds.norm(p=2, dim=-1, keepdim=True) # normalize
        class_text_embeds[text] = text_embeds.squeeze(0)  # Save class embedding

100%|██████████| 10/10 [00:00<00:00, 12.90it/s]


In [8]:
print(class_text_embeds.keys())

dict_keys(['A photo of airplane', 'A photo of automobile', 'A photo of bird', 'A photo of cat', 'A photo of deer', 'A photo of dog', 'A photo of frog', 'A photo of horse', 'A photo of ship', 'A photo of truck'])


In [9]:
n_classes = len(test_data.classes)
n_samples = len(test_data)  # total number of samples
res_img = torch.empty((n_samples, 512))  # Preallocate memory
res_text = torch.empty((n_samples, 512))
index = 0

loader = DataLoader(dataset=test_data, batch_size = 32, shuffle=False) 

for batch in tqdm(loader): 
    images, texts, _ = batch
    # Retrieve embeddings using list comprehension
    text_embeds = [class_text_embeds.get(text) for text in texts]
    # Convert to tensor if needed
    text_embeds = torch.tensor(np.array(text_embeds))
    
    img_input = processor(text=None, images=images, return_tensors='pt', padding=True)
    with torch.no_grad(): 
        image_embeds = model.get_image_features(**img_input)
        image_embeds = image_embeds / image_embeds.norm(p=2, dim=-1, keepdim=True)
    
    # print(image_embeds.shape)
    batch_size = image_embeds.size(0)
    res_img[index:index + batch_size] = image_embeds
    res_text[index:index + batch_size] = text_embeds
    index += batch_size
    # break

100%|██████████| 313/313 [11:20<00:00,  2.17s/it]


In [10]:
# with open('data/cifar10/embedded/cifar10-image-embeds.pt','wb') as f:
#     torch.save(res_img,f)

In [11]:
# with open('data/cifar10/embedded/cifar10-text-embeds.pt','wb') as f:
#     torch.save(res_text,f)

In [6]:
# test 
img_embedding, text_embedding = torch.load('data/cifar10/embedded/cifar10-image-embeds.pt'), torch.load('data/cifar10/embedded/cifar10-text-embeds.pt')

In [7]:
print(img_embedding) 
print(text_embedding)

tensor([[ 1.9753e-02,  4.1835e-03, -3.7525e-02,  ...,  4.4831e-02,
          2.6137e-03,  1.9466e-02],
        [ 4.3587e-02, -2.7502e-03, -6.0672e-03,  ...,  4.9582e-02,
         -3.0790e-03, -2.2833e-02],
        [ 1.2498e-02, -7.4182e-05, -2.1343e-02,  ...,  5.3345e-02,
          1.1302e-03,  4.5888e-03],
        ...,
        [ 3.1369e-04, -1.4004e-02, -3.4491e-02,  ...,  6.4544e-02,
          1.9953e-02,  5.4879e-03],
        [ 4.4128e-02,  1.3703e-02, -2.7129e-03,  ...,  3.4298e-02,
         -1.2507e-03, -1.3165e-03],
        [ 1.3879e-02,  1.1702e-03, -3.3632e-03,  ...,  5.1820e-02,
          1.1141e-02,  1.3778e-02]])
tensor([[ 0.0120,  0.0053, -0.0148,  ..., -0.0423, -0.0423,  0.0091],
        [ 0.0077,  0.0015, -0.0056,  ..., -0.0358,  0.0025, -0.0322],
        [ 0.0077,  0.0015, -0.0056,  ..., -0.0358,  0.0025, -0.0322],
        ...,
        [ 0.0044,  0.0226, -0.0318,  ..., -0.0422, -0.0267,  0.0169],
        [ 0.0058,  0.0140,  0.0004,  ..., -0.0032, -0.0385, -0.0088],
     

In [16]:
for classs, class_embed in class_text_embeds.items(): 
    cnt = 0
    for tensor in text_embedding:
        if torch.allclose(tensor, class_embed):
            cnt += 1  
    
    print(f"{classs}: {cnt}")

A photo of airplane: 1000
A photo of automobile: 1000
A photo of bird: 1000
A photo of cat: 1000
A photo of deer: 1000
A photo of dog: 1000
A photo of frog: 1000
A photo of horse: 1000
A photo of ship: 1000
A photo of truck: 1000


In [17]:
test_data.classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [18]:
for x in test_data.class_to_idx: 
    print(x, test_data.class_to_idx[x])

airplane 0
automobile 1
bird 2
cat 3
deer 4
dog 5
frog 6
horse 7
ship 8
truck 9


# 3: Test zero-shot inference

In [8]:
from sklearn.metrics import accuracy_score
from optimal_transport.adapters.info import InfoOT
from optimal_transport.adapters.emd import EMD
from optimal_transport.adapters.entropic import EntropicOT
from optimal_transport.adapters.low_rank import LrSinkhornOT
from optimal_transport.adapters.kpg.kp import KPGRLKP
from optimal_transport.adapters.factored.fot import FOT
from optimal_transport.adapters.factored.kfot import KFOT
from optimal_transport.adapters.factored.lot import LOT
import torch.nn.functional as F
import ot as OT

In [20]:
# baseline: cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
P = cosine_similarity(img_embedding, text_embedding)
res = [test_data.targets[x] for x in P.argmax(1)]
print(res)
print(test_data.targets)

print(accuracy_score(test_data.targets, res))


[3, 8, 8, 0, 6, 5, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 2, 4, 1, 9, 5, 7, 6, 2, 6, 0, 9, 3, 8, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 5, 4, 7, 3, 6, 3, 6, 2, 1, 2, 0, 7, 2, 6, 8, 8, 9, 2, 9, 3, 3, 8, 8, 1, 1, 7, 7, 7, 0, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 0, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 3, 3, 6, 3, 8, 7, 6, 2, 5, 2, 0, 9, 0, 0, 0, 5, 2, 9, 5, 4, 2, 1, 7, 6, 8, 7, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 0, 0, 2, 5, 2, 2, 3, 8, 6, 3, 4, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 2, 7, 9, 8, 2, 7, 6, 9, 5, 3, 9, 2, 4, 7, 6, 5, 1, 5, 8, 8, 0, 4, 8, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 7, 0, 3, 0, 0, 9, 8, 1, 5, 7, 0, 8, 2, 4, 7, 0, 2, 5, 6, 3, 8, 3, 0, 3, 7, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 3, 6, 0, 0, 6, 6, 6, 7, 2, 6, 1, 8, 2, 1, 6, 8, 6, 8, 2, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 7, 1, 7, 5, 4, 6, 1, 1, 3, 

In [22]:
# Fused-Gromov softmax
def fgwi_softmax(image_features, text_features, a1, a2, reg, n):
    """
    image_features: image features
    text_features: text features
    a1: regularization term for scaling the KL divergence term 1
    a2: regularization term for scaling the KL divergence term 2
    reg: regularization parameter > 0
    n: number of iterations for adjusting the structural cost matrix
    """

    # KL term 1: compute the self-similarity matrix for image features
    sigma_1 = image_features @ image_features.t()
    # KL term 2: compute the self-similarity matrix for text features
    sigma_2 = text_features @ text_features.t()

    # init cost matrix
    C = 1.0 - image_features @ text_features.t()
    # init transport plan
    P = F.softmax(-C / reg)

    for _ in tqdm(range(n)):
        C = C.cuda()
        C = C - a1 * sigma_1.cuda() @ P.cuda() @ sigma_2.cuda() * a2
        P = F.softmax(-C/ reg).cpu()
        C = C.cpu()
    return P.cpu()

transport_plan = fgwi_softmax(img_embedding, text_embedding, 1e-4,1e-4,0.01,100)
res = [test_data.targets[x] for x in transport_plan.argmax(1)]

print(accuracy_score(test_data.targets,res))

100%|██████████| 100/100 [03:57<00:00,  2.37s/it]


0.8891


In [40]:
torch.cuda.empty_cache()

In [10]:
def fgw_pgrad(image_f, text_f, a1, a2, reg, N):    
    # image f:Image features
    # text f:Text features
    # a1:Regularization term for scaling the KL divergence term 1
    # a2:Regularization term for scaling the KL divergence term 2
    # reg:Regularization term > 0
    # N:Max number of iterations for adjusting the structural cost matrix
    # KL term 1: Compute the self-similarity matrix for image features
    sigma_1 = image_f @ image_f.t()
    # KL term 2: Compute the self-similarity matrix for text features
    sigma_2 = text_f @ text_f.t()
    # Construct initial cost matrix based on the negative dot product (similarity) between image and text features
    C = 1.0 - image_f @ text_f.t()
    # Initialize the transport plan P
    P = F.softmax(-C)
    # Iteratively refine the structural cost matrix
    for i in tqdm(range(N)):
        # Adjust the cost matrix C
        C = C.cuda()
        grad = C - a1 * sigma_1.cuda() @ P.cuda() @ sigma_2.cuda() * a2
        K = P.cuda() * torch.exp(-grad/reg)
        del grad
        P = K / K.sum(dim=1,keepdim=True) # Row normalization
        P = P.cpu()
        del K
        C = C.cpu()
        sigma_1 = sigma_1.cpu()
        sigma_2 = sigma_2.cpu()        
    # Return transport plan P
    return P

P = fgw_pgrad(img_embedding,text_embedding,0.01,0.01,0.01,5)
res = [test_data.targets[x] for x in P.argmax(1)]

accuracy_score(test_data.targets,res)

100%|██████████| 5/5 [00:09<00:00,  1.95s/it]


0.7286

In [10]:
# partial OT
def partial_ot(image_features, text_features, reg, n, num_class):
    """
    image_features: image features
    text_features: text features
    reg: regularization parameter > 0
    n: number of iterations for adjusting the structural cost matrix
    num_class: number of prediction classes
    """

    P = torch.exp(image_features @ text_features.t()) / reg
    m = torch.full((num_class,), 1/num_class)

    for _ in range(n):
        row_P = P.sum(dim=1)
        print(row_P.shape)
        scaling = torch.max(row_P)
        P = torch.div(P, scaling)
        P *= m/P.sum(dim=1)
    return P

P = partial_ot(img_embedding,text_embedding, reg = 0.1, n = 100, num_class = 10)
res = [test_data.targets[x] for x in P.argmax(1)]

accuracy_score(test_data.targets,res)

torch.Size([10000])


RuntimeError: The size of tensor a (10) must match the size of tensor b (10000) at non-singleton dimension 0

In [14]:
ot = LrSinkhornOT()
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(),np.ones(img_embedding.shape[0])/img_embedding.shape[0],np.ones(text_embedding.shape[0])/text_embedding.shape[0])
res = [test_data.targets[x] for x in ot.P_.argmax(1)]

print(accuracy_score(test_data.targets,res))

0.2919


In [12]:
ot = EntropicOT()
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(), np.ones(img_embedding.shape[0])/img_embedding.shape[0], np.ones(text_embedding.shape[0])/text_embedding.shape[0])
res = [test_data.targets[x] for x in ot.P_.argmax(1)]

print(accuracy_score(test_data.targets,res))

0.8972


In [8]:
ot = InfoOT(num_iters=10)
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(),np.ones(img_embedding.shape[0])/img_embedding.shape[0],np.ones(text_embedding.shape[0])/text_embedding.shape[0])
res = [test_data.targets[x] for x in ot.P_.argmax(1)]

print(accuracy_score(test_data.targets,res))

0.0844


In [22]:
ot = EMD()
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(),np.ones(img_embedding.shape[0])/img_embedding.shape[0],np.ones(text_embedding.shape[0])/text_embedding.shape[0])
res = ot.P_.argmax(axis=1)

print(accuracy_score(test_data.targets,res))

TypeError: emd() got an unexpected keyword argument 'maxItermax'

In [13]:
# KFOT
import torch 
from collections import defaultdict
img_embedding_idx = defaultdict(lambda : [])

# Compute dictionary of img_embedding index list. The index is clustered according to the corresponding text embedding
id = -1
set_text_embedding = []
for idx in range(text_embedding.shape[0]):
    for i, a in enumerate(set_text_embedding):
        if torch.all(text_embedding[idx] == a):
            img_embedding_idx[i].append(idx)
            break
    else:
        set_text_embedding.append(text_embedding[idx])
        id += 1
        img_embedding_idx[id].append(idx)

K = []
# Compute centroid of each cluster
for key in img_embedding_idx: 
    idx_list = img_embedding_idx[key] 
    cluster = img_embedding[idx_list] 
    centroid = torch.mean(cluster, dim=0)
    dist = torch.tensor([torch.dist(centroid, point) for point in cluster])
    nearest_centroid_point = cluster[torch.argmin(dist)]
    nearest_centroid_point_idx = int(torch.where(img_embedding==nearest_centroid_point)[0][0])
    K.append((nearest_centroid_point_idx, key))

ot = KFOT(n_free_anchors=6)
a = np.ones(img_embedding.shape[0]) / img_embedding.shape[0]  
b = np.ones(text_embedding.shape[0]) / text_embedding.shape[0]  
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(), a, b,
            K = K, ys = a, yt = b)

res = [test_data.targets[x] for x in ot.P_.argmax(1)]

print(accuracy_score(test_data.targets,res))

0.4814


In [8]:
ot = FOT(solver_kwargs={
            "r": 3, "reg": 0.01, "stopThr": 1e-7,
            "numItermax": 10000, "method": "sinkhorn_log"
        })
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(),np.ones(img_embedding.shape[0])/img_embedding.shape[0],np.ones(text_embedding.shape[0])/text_embedding.shape[0])
res = [test_data.targets[x] for x in ot.P_.argmax(1)]
print(accuracy_score(test_data.targets,res))

0.2832


In [11]:
batch_size_lot = 128
predictions = []

ot = LOT(distance=OT.dist, n_source_anchors=100, n_target_anchors=100)


for i in range(0, img_embedding.shape[0], batch_size_lot):
    img_batch = img_embedding[i:i+batch_size_lot].numpy()
    ot.fit(img_batch, text_embedding.numpy(),
                np.ones(img_batch.shape[0]) / img_batch.shape[0],
                np.ones(text_embedding.shape[0]) / text_embedding.shape[0])
    
    batch_res = [test_data.targets[x] for x in ot.P_.argmax(1)]
    predictions.extend(batch_res)

print(accuracy_score(test_data.targets[:len(predictions)], predictions))

KeyboardInterrupt: 